In [1]:
# Import libraries
import os
import sys
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file
price_data = pd.read_csv('./data/CCL_historical_data.csv')
price_data

,datetime,o,h,l,c,v,readable_time
0,1571228400000,42.49,42.4900,42.49,42.49,270,2019-10-16 07:20:00
1,1571229300000,42.50,42.5000,42.50,42.50,400,2019-10-16 07:35:00
2,1571232300000,42.69,42.7300,42.69,42.73,3139,2019-10-16 08:25:00
3,1571232600000,42.65,42.7200,42.50,42.68,102539,2019-10-16 08:30:00
4,1571232900000,42.68,42.8622,42.62,42.82,45839,2019-10-16 08:35:00
...,...,...,...,...,...,...,...
25444,1593646500000,16.45,16.4800,16.44,16.48,1584,2020-07-01 18:35:00
25445,1593646800000,16.48,16.4800,16.44,16.48,1100,2020-07-01 18:40:00
25446,1593647100000,16.48,16.4900,16.48,16.49,2193,2020-07-01 18:45:00
25447,1593647400000,16.48,16.4800,16.45,16.47,3243,2020-07-01 18:50:00


In [2]:
# Add a "Change in price" column
price_data['change_in_price'] = price_data['c'].diff()
price_data.head()

,datetime,o,h,l,c,v,readable_time,change_in_price
0,1571228400000,42.49,42.4900,42.49,42.49,270,2019-10-16 07:20:00,NaN
1,1571229300000,42.50,42.5000,42.50,42.50,400,2019-10-16 07:35:00,0.01
2,1571232300000,42.69,42.7300,42.69,42.73,3139,2019-10-16 08:25:00,0.23
3,1571232600000,42.65,42.7200,42.50,42.68,102539,2019-10-16 08:30:00,-0.05
4,1571232900000,42.68,42.8622,42.62,42.82,45839,2019-10-16 08:35:00,0.14


In [3]:
# Calculate 20 EMA (short term)
ema_20 = price_data['c'].transform(lambda x: x.ewm(span = 20).mean())
price_data['EMA_20'] = ema_20

# Calculate 200 EMA (long term)
ema_200 = price_data['c'].transform(lambda x: x.ewm(span = 200).mean())
price_data['EMA_200'] = ema_200

#  Calculate MACD 
ema_26 = price_data['c'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = price_data['c'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

ema_9_macd = macd.ewm(span = 9).mean()

# Store the data in the data fram
price_data['MACD'] = macd
price_data['MACD_EMA'] = ema_9_macd
price_data['MACD_Diff'] = macd - ema_9_macd

price_data.tail(50)

,datetime,o,h,l,c,v,readable_time,change_in_price,EMA_20,EMA_200,MACD,MACD_EMA,MACD_Diff
25399,1593632700000,16.3700,16.4100,16.3400,16.3900,252622,2020-07-01 14:45:00,0.0250,16.362358,16.455167,-0.010038,-0.027756,0.017718
25400,1593633000000,16.3900,16.3900,16.2950,16.3450,382601,2020-07-01 14:50:00,-0.0450,16.360705,16.454070,-0.010700,-0.024345,0.013644
25401,1593633300000,16.3400,16.3700,16.3050,16.3600,652604,2020-07-01 14:55:00,0.0150,16.360638,16.453134,-0.009900,-0.021456,0.011555
25402,1593633600000,16.3600,16.3900,16.3001,16.3100,390074,2020-07-01 15:00:00,-0.0500,16.355815,16.451710,-0.013150,-0.019795,0.006645
25403,1593633900000,16.3600,16.3600,16.3200,16.3600,1206,2020-07-01 15:05:00,0.0500,16.356214,16.450798,-0.011557,-0.018147,0.006590
25404,1593634200000,16.3601,16.4000,16.3601,16.3700,3576,2020-07-01 15:10:00,0.0100,16.357527,16.449994,-0.009380,-0.016394,0.007014
25405,1593634500000,16.3600,16.3600,16.3500,16.3500,4325,2020-07-01 15:15:00,-0.0200,16.356810,16.448999,-0.009162,-0.014947,0.005785
25406,1593634800000,16.3600,16.3601,16.3600,16.3600,6026,2020-07-01 15:20:00,0.0100,16.357114,16.448113,-0.008090,-0.013576,0.005486
25407,1593635100000,16.3600,16.3600,16.3600,16.3600,1274,2020-07-01 15:25:00,0.0000,16.357388,16.447236,-0.007157,-0.012292,0.005135
25408,1593635400000,16.3600,16.3700,16.3600,16.3600,500,2020-07-01 15:30:00,0.0000,16.357637,16.446368,-0.006345,-0.011103,0.004757
